In [54]:
#Importo las librerías necesarias
import pandas as pd

In [55]:
#Cargo el dataset de lesiones de 1951-2023
print(f"Cargando archivo original...")
try:
    df_hasta_2023 = pd.read_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA Player Injury Stats(1951 - 2023).csv")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo 'C:\\Users\\FranciscoJH\\Downloads\\NBA Player Injury Stats(1951 - 2023).csv'. Asegúrate de que esté en la misma carpeta.")
    exit()

print(f"Datos cargados. Se encontraron {len(df)} filas.")

Cargando archivo original...
Datos cargados. Se encontraron 16873 filas.


In [56]:
# Conversión de Fecha
# Nos aseguramos de que la columna 'Date' sea de tipo fecha
columna_fecha = 'Date'
# 'errors=coerce' convertirá cualquier fecha rota en 'NaT' (Not a Time)
print(f"Convirtiendo la columna '{columna_fecha}' a formato de fecha...")
df_hasta_2023[columna_fecha] = pd.to_datetime(df_hasta_2023[columna_fecha], errors='coerce')

Convirtiendo la columna 'Date' a formato de fecha...


In [57]:
# Eliminamos filas donde la fecha no se pudo leer
df_hasta_2023.dropna(subset=[columna_fecha], inplace=True)

In [58]:
# Extraemos el año de la fecha y filtramos
print("Filtrando datos entre 2010 y 2023...")
df_filtrado_hasta_2023 = df_hasta_2023[(df_hasta_2023[columna_fecha].dt.year >= 2010) & (df_hasta_2023[columna_fecha].dt.year <= 2023)]
#Además elimino la primer columna que almacena el ID del registro, ya que el dataset de lesiones de 2016-2025 no la tiene
df_filtrado_hasta_2023 = df_filtrado_hasta_2023.iloc[:, 1:]

Filtrando datos entre 2010 y 2023...


In [59]:
#Guardo el dataset filtrado
print(f"Filtrado completo. Se encontraron {len(df_filtrado_hasta_2023)} filas.")
df_filtrado_hasta_2023.to_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_Filtrado_2023.csv", index=False)

Filtrado completo. Se encontraron 22998 filas.


In [60]:
#Ahora cargo el dataset de lesiones de 2016-2025
print(f"Cargando archivo original...")
try:
    df_hasta_2025 = pd.read_csv("C:\\Users\\FranciscoJH\\Downloads\\injury_data.csv")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo 'C:\\Users\\FranciscoJH\\Downloads\\injury_data.csv'. Asegúrate de que esté en la misma carpeta.")
    exit()

print(f"Datos cargados. Se encontraron {len(df_hasta_2025)} filas.")

Cargando archivo original...
Datos cargados. Se encontraron 16873 filas.


In [61]:
# Conversión de Fecha
# Nos aseguramos de que la columna 'Date' sea de tipo fecha
columna_fecha = 'Date'
# 'errors=coerce' convertirá cualquier fecha rota en 'NaT' (Not a Time)
print(f"Convirtiendo la columna '{columna_fecha}' a formato de fecha...")
df_hasta_2025[columna_fecha] = pd.to_datetime(df_hasta_2025[columna_fecha], errors='coerce')

Convirtiendo la columna 'Date' a formato de fecha...


In [62]:
# Eliminamos filas donde la fecha no se pudo leer
df_hasta_2025.dropna(subset=[columna_fecha], inplace=True)

In [63]:
# Extraemos el año de la fecha y filtramos
print("Filtrando datos entre 2023 y 2025...")
df_filtrado_hasta_2025 = df_hasta_2025[(df_hasta_2025[columna_fecha].dt.year >= 2023) & (df_hasta_2025[columna_fecha].dt.year <= 2025)]

Filtrando datos entre 2023 y 2025...


In [64]:
#Guardo el dataset filtrado
print(f"Filtrado completo. Se encontraron {len(df_filtrado_hasta_2025)} filas.")
df_filtrado_hasta_2025.to_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_Filtrado_2025.csv", index=False)

Filtrado completo. Se encontraron 3955 filas.


In [65]:
#Ahora juntos los dos datasets
df_total = pd.concat([df_filtrado_hasta_2023, df_filtrado_hasta_2025], ignore_index=True)

#Elijo las columnas clave para eliminar los duplicados de año 2023
columnas_clave = ['Date', 'Team', 'Acquired', 'Relinquished', 'Notes']

#Elimino duplicados basándome en las columnas clave
#'keep=' controla cuál duplicado se queda:
#'last':  Se queda con el último registro de los duplicados que ve.
lesiones_csv = df_total.drop_duplicates(subset=columnas_clave, keep='last')

#Guardo el dataset final
lesiones_csv.to_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_2010_2025.csv", index=False)
print(f"¡Éxito! Archivo final guardado en 'C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_2010_2025.csv'")

¡Éxito! Archivo final guardado en 'C:\Users\FranciscoJH\Downloads\NBA_Player_Injury_Stats_2010_2025.csv'
